# Import data and format transformation

In [26]:
# import receipt data from json file
import pandas as pd
import numpy as np

receipts = pd.read_json('data/receipts.json', lines=True)
receipts.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b


### Replace the dictionary columns with its values

In [27]:
# handle date format and return as string
def handle_dateFormat(x):
    values = x.apply(lambda x: x['$date'] if isinstance(x, dict) else pd.NaT)
    dt_values = pd.to_datetime(values, unit='ms')

    return dt_values.dt.strftime('%Y-%m-%d %H:%M:%S')

In [28]:
receipts['createDate'] = handle_dateFormat(receipts['createDate'])
receipts['dateScanned'] = handle_dateFormat(receipts['dateScanned'])
receipts['finishedDate'] = handle_dateFormat(receipts['finishedDate'])
receipts['modifyDate'] = handle_dateFormat(receipts['modifyDate'])
receipts['pointsAwardedDate'] = handle_dateFormat(receipts['pointsAwardedDate'])
receipts['purchaseDate'] = handle_dateFormat(receipts['purchaseDate'])

# handle _id column
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])
receipts.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaN,2021-01-03 15:25:42,NaN,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b


#### Since there are lots of items stored on the rewardlist column, I create another table called rewardList to store the price, brand etc for each reward items.  

In [29]:
#filter all the rows that are rewardsReceiptStatus is not FINISHED and purchasedItemCount > 0
receipts_finished = receipts[(receipts['rewardsReceiptStatus'] == 'FINISHED') & (receipts['purchasedItemCount'] > 0)]
receipts_finished.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6


In [30]:
# create a new dataframe to store the information in the rewardsReceiptItemList column that divide every item into a row
receipts_rewardList = pd.DataFrame(columns=['receipt_id', 'barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview', 'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased', 'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice', 'userFlaggedQuantity', 'needsFetchReviewReason', 'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup', 'rewardsProductPartnerId', 'userFlaggedDescription', 'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode', 'competitorRewardsGroup', 'discountedItemPrice', 'originalReceiptItemText', 'itemNumber', 'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice', 'competitiveProduct', 'originalFinalPrice', 'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon', 'metabriteCampaignId', 'originalMetaBriteNeedsFetchReview', 'originalMetaBritePartnerItemId', 'pointsAwardedDate', 'rewardsReceiptStatus', 'totalSpent', 'userId', 'rewardsReceiptItemList_id'])

# divide the information in the rewardsReceiptItemList column into the new dataframe with corresponding columns
for i in range(len(receipts_finished)):
    for j in range(len(receipts_finished['rewardsReceiptItemList'].iloc[i])):
        receipts_rewardList = receipts_rewardList.append(receipts_finished['rewardsReceiptItemList'].iloc[i][j], ignore_index=True)
        receipts_rewardList['receipt_id'].iloc[-1] = receipts_finished['_id'].iloc[i]

receipts_rewardList.head(3)

# save the dataframe into a csv file
receipts_rewardList.to_csv('data/receipts_rewardList.csv', index=False)

In [31]:
# drop the rewardsReceiptItemList column in the receipt dataframe
receipts = receipts.drop(columns=['rewardsReceiptItemList'])

# save the dataframe into a csv file 
receipts.to_csv('data/receipts_updated.csv', index=False)

## Load brands data

In [32]:
# import  brands data from json file 
brands = pd.read_json('data/brands.json', lines=True, dtype={'topBrand': bool})

# extract values from the _id column
brands['_id'] = brands['_id'].apply(lambda x: x['$oid'])

# extract values from the cpg column and store them into a new column called cpg_id and cpg_ref
brands['cpg_id'] = brands['cpg'].apply(lambda x: x['$id']['$oid'])
brands['cpg_ref'] = brands['cpg'].apply(lambda x: x['$ref'])

# drop the cpg column
brands = brands.drop(columns=['cpg'])


# find out rows that have duplicated values
brands[brands['barcode'].duplicated(keep=False)]

# save the dataframe into a csv file 
brands.to_csv('data/brands_updated.csv', index=True)
brands.head(10)

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,False,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,False,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,False,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,False,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs
5,601ac142be37ce2ead43755b,511111719885,Baking,BAKING,test brand @1612366146091,False,TEST BRANDCODE @1612366146091,601ac142be37ce2ead437559,Cogs
6,601ac142be37ce2ead43755c,511111219897,Baking,BAKING,test brand @1612366146133,False,TEST BRANDCODE @1612366146133,601ac142be37ce2ead437559,Cogs
7,5cdad0f5166eb33eb7ce0faa,511111104810,Condiments & Sauces,NaN,J.L. Kraft,True,J.L. KRAFT,559c2234e4b06aca36af13c6,Cogs
8,5ab15636e4b0be0a89bb0b07,511111504412,Canned Goods & Soups,NaN,Campbell's Home Style,False,CAMPBELLS HOME STYLE,5a734034e4b0d58f376be874,Cogs
9,5c408e8bcd244a1fdb47aee7,511111504788,Baking,NaN,test,True,TEST,59ba6f1ce4b092b29c167346,Cogs


## Load users data

In [33]:
# import  users data from json file
users = pd.read_json('data/users.json', lines=True)

# extract values from the _id column
users['_id'] = users['_id'].apply(lambda x: x['$oid'])

# use the above function to extract values from all the date columns
users['createdDate'] = handle_dateFormat(users['createdDate'])
users['lastLogin'] = handle_dateFormat(users['lastLogin'])

# save the updated information into a csv file
users.to_csv('data/users_updated.csv', index=True)
users.head(10)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
5,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03 15:25:28,2021-01-03 15:25:28,consumer,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,True,2021-01-03 15:24:39,2021-01-03 15:24:39,consumer,Email,WI
8,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,True,2021-01-03 15:25:37,2021-01-03 15:25:37,consumer,Email,WI


# Connect to a SQL server and answer questions

In [34]:
# connect to a MYSQL database
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="fetchdata"
)

mycursor = mydb.cursor()

#mycursor.execute("set global local_infile = 1;")
mycursor.execute("DROP DATABASE IF EXISTS fetch_dataChallenge")
mycursor.execute("CREATE DATABASE fetch_dataChallenge")
 

### Import the cleande csv file into Mysql database

In [35]:
# create updated brands tables in the database 
mycursor.execute("USE fetch_dataChallenge")
mycursor.execute("DROP TABLE IF EXISTS brands_updated")
mycursor.execute("CREATE TABLE brands_updated (id VARCHAR(255), barcode VARCHAR(255), category VARCHAR(255), categoryCode VARCHAR(255), cpg_id VARCHAR(255), cpg_ref VARCHAR(255), name VARCHAR(255), topBrand VARCHAR(255), \
                 PRIMARY KEY (id))")

# change the data type of the columns to string type
brands['_id'] = brands['_id'].astype(str)  #int(brands['_id'])
brands['barcode'] = brands['barcode'].astype(str) 
brands['topBrand'] = brands['topBrand'].astype(str)

# Insert row by row to Table with its corresponding columns from the receipts_updated dataframe
for i in range(len(brands)):
    sql = "INSERT INTO brands_updated (id, barcode, category, categoryCode, cpg_id, cpg_ref, name, topBrand) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    val = (brands['_id'].iloc[i], brands['barcode'].iloc[i], brands['category'].iloc[i], brands['categoryCode'].iloc[i], brands['cpg_id'].iloc[i], brands['cpg_ref'].iloc[i], brands['name'].iloc[i], brands['topBrand'].iloc[i])
    mycursor.execute(sql, val)
    mydb.commit()


# print the first 10 rows of the brands table
mycursor.execute("SELECT * FROM brands_updated LIMIT 10")
myresult = mycursor.fetchall()

for x in myresult:
  print(x)

('5332f5ebe4b03c9a25efd0a7', '511111304050', None, None, '5332f5ebe4b03c9a25efd0a8', 'Cpgs', 'Monster', 'True')
('5332f5f2e4b03c9a25efd0a9', '511111804048', None, None, '5332f5f2e4b03c9a25efd0aa', 'Cpgs', 'Eggo', 'True')
('5332f5f2e4b03c9a25efd0ab', '511111604037', None, None, '5332f5f3e4b03c9a25efd0ac', 'Cpgs', 'Our Family', 'True')
('5332f5f3e4b03c9a25efd0ad', '511111104025', None, None, '5332f5f3e4b03c9a25efd0ae', 'Cpgs', 'Gree Giant', 'True')
('5332f5f4e4b03c9a25efd0af', '511111904014', None, None, '5332f5f2e4b03c9a25efd0aa', 'Cpgs', 'Frosted Mini-Wheats', 'True')
('5332f5f5e4b03c9a25efd0b0', '511111404002', None, None, '5332f5f3e4b03c9a25efd0ae', 'Cpgs', 'Betty Crocker', 'True')
('5332f5f5e4b03c9a25efd0b1', '511111203995', None, None, '5332f5ebe4b03c9a25efd0a8', 'Cpgs', 'Minute Maid', 'True')
('5332f5f6e4b03c9a25efd0b2', '511111703983', None, None, '5332f5ebe4b03c9a25efd0a8', 'Cpgs', 'Coca-Cola', 'True')
('5332f5fbe4b03c9a25efd0b9', '511111503972', 'Beverages', None, '5332f5fbe4b0

In [36]:
# Create users table in the database
mycursor.execute("USE fetch_dataChallenge")
mycursor.execute("DROP TABLE IF EXISTS users_updated")
mycursor.execute("CREATE TABLE users_updated (id VARCHAR(255), active VARCHAR(255), created_date VARCHAR(255), last_login VARCHAR(255), role VARCHAR(255), signup_source VARCHAR(255), state VARCHAR(255))")

users_updated = pd.read_csv('data/users_updated.csv').drop(columns=['Unnamed: 0'])

# use this for loop to insert row by row to Table 
for index, row in users_updated.iterrows():
    mycursor.execute("INSERT INTO fetch_dataChallenge.users_updated (id, active, created_date, last_login, role, signup_source, state)  VALUES (%s, %s, %s, %s, %s, %s, %s) ", tuple(row) )
mydb.commit()

# print the first 10 rows of the users table
mycursor.execute("SELECT * FROM users_updated LIMIT 10")
myresult = mycursor.fetchall()

for x in myresult:
    print(x)


('5ff1e194b6a9d73a3a9f1052', '1', '2021-01-03 15:24:04', '2021-01-03 15:25:37', 'consumer', 'Email', 'WI')
('5ff1e194b6a9d73a3a9f1052', '1', '2021-01-03 15:24:04', '2021-01-03 15:25:37', 'consumer', 'Email', 'WI')
('5ff1e194b6a9d73a3a9f1052', '1', '2021-01-03 15:24:04', '2021-01-03 15:25:37', 'consumer', 'Email', 'WI')
('5ff1e1eacfcf6c399c274ae6', '1', '2021-01-03 15:25:30', '2021-01-03 15:25:30', 'consumer', 'Email', 'WI')
('5ff1e194b6a9d73a3a9f1052', '1', '2021-01-03 15:24:04', '2021-01-03 15:25:37', 'consumer', 'Email', 'WI')
('5ff1e194b6a9d73a3a9f1052', '1', '2021-01-03 15:24:04', '2021-01-03 15:25:37', 'consumer', 'Email', 'WI')
('5ff1e1e8cfcf6c399c274ad9', '1', '2021-01-03 15:25:28', '2021-01-03 15:25:28', 'consumer', 'Email', 'WI')
('5ff1e1b7cfcf6c399c274a5a', '1', '2021-01-03 15:24:39', '2021-01-03 15:24:39', 'consumer', 'Email', 'WI')
('5ff1e194b6a9d73a3a9f1052', '1', '2021-01-03 15:24:04', '2021-01-03 15:25:37', 'consumer', 'Email', 'WI')
('5ff1e1f1cfcf6c399c274b0b', '1', '20

In [37]:
# create updated receipts tables in the database 
mycursor.execute("USE fetch_dataChallenge")
mycursor.execute("DROP TABLE IF EXISTS receipts_updated")
mycursor.execute("CREATE TABLE receipts_updated (id VARCHAR(255), bonusPointsEarned FLOAT, bonusPointsEarnedReason VARCHAR(255), create_date VARCHAR(255), date_scanned VARCHAR(255), finished_date VARCHAR(255), modify_date VARCHAR(255), points_awarded_date VARCHAR(255), points_earned FLOAT, purchase_date VARCHAR(255), purchased_item_count FLOAT, rewards_receipt_status VARCHAR(255), total_spent FLOAT, user_id VARCHAR(255), \
                 PRIMARY KEY (id))")#, FOREIGN KEY (user_id) REFERENCES users_updated(id))

receipts_updated = receipts  #####pd.read_csv('receipts_updated.csv')

# use this for loop to insert row by row to Table with its corresponding columns from the receipts_updated dataframe
for index, row in receipts_updated.iterrows():
    mycursor.execute("INSERT INTO fetch_dataChallenge.receipts_updated (id, bonusPointsEarned, bonusPointsEarnedReason, create_date, date_scanned, finished_date, modify_date, points_awarded_date, points_earned, purchase_date, purchased_item_count, rewards_receipt_status, total_spent, user_id)  VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s,%s,%s,%s) ", tuple(row) )
mydb.commit()


# print the first 10 rows of the receipts table
mycursor.execute("SELECT * FROM receipts_updated LIMIT 10")
myresult = mycursor.fetchall()

for x in myresult:
  print(x)

('5f9c74f70a7214ad07000037', 750.0, 'Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)', '2020-10-30 20:17:59', '2020-10-30 20:17:59', None, '2021-01-03 15:24:54', '2020-10-30 20:18:00', 750.0, '2020-10-29 20:17:59', 11.0, 'REJECTED', 1.0, '5f9c74f7c88c1415cbddb839')
('5f9c74f90a7214ad07000038', 300.0, 'Receipt number 4 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)', '2020-10-30 20:18:01', '2020-10-30 20:18:01', '2021-01-03 15:39:55', '2021-01-03 15:39:55', '2021-01-03 15:39:55', 389.2, '2020-10-29 20:18:01', 6.0, 'FINISHED', 14.0, '5f9c74e3f1937815bd2c1d73')
('5fa5ad370a720f05ef000089', 750.0, 'Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)', '2020-11-06 20:08:23', '2020-11-06 20:08:23', '2021-01-05 20:53:40', '2021-01-05 20:53:40', '2021-01-05 20:53:40', 9449.8, '2020-11-05 20:08:23', 11.0, 'FINISHED', 291.0, '5fa5ad376a26f611e71ab5ef')
('5fa5b0ca0a720f05ef0000bf', 100.0, 'Receipt number 6 com

In [38]:
table_name = 'receipts_rewardList'
database_name = 'fetch_dataChallenge'
mycursor.execute("USE fetch_dataChallenge")
mycursor.execute("DROP TABLE IF EXISTS receipts_rewardList")
# create the table that has the same columns name as the receipts_rewardList dataframe
mycursor.execute(f"CREATE TABLE {database_name}.{table_name} ({', '.join([f'{col} VARCHAR(255)' for col in receipts_rewardList.columns])})")

receipts_rewardList = pd.read_csv('data/receipts_rewardList.csv')

# read the barcode columns as string type and drop the fraction part
receipts_rewardList['barcode'] = receipts_rewardList['barcode'].astype(str).str.split('.').str[0]

# Create a cursor to execute SQL queries
mycursor = mydb.cursor()

# Iterate over each row in the DataFrame
for index, row in receipts_rewardList.iterrows():
    row = row.where(pd.notnull(row), None)  # Convert missing values to None

    # Insert the row into the SQL table
    query = f"INSERT INTO {database_name}.{table_name} ({', '.join(receipts_rewardList.columns)}) VALUES ({', '.join(['%s'] * len(receipts_rewardList.columns))})"
    mycursor.execute(query, tuple(row))

# Commit the changes and close the database connection
mydb.commit()


# print the first 10 rows of the brands table
mycursor.execute("SELECT * FROM receipts_rewardList LIMIT 10")
myresult = mycursor.fetchall()

for x in myresult:
  print(x)

('5ff1e1eb0a720f0523000575', '4011', 'ITEM NOT FOUND', '26.0', '26.0', '0', '1', '1', '5.0', '4011.0', '1', '26.0', '5.0', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
('5ff1e1bb0a720f052300056b', '4011', 'ITEM NOT FOUND', '1.0', '1.0', None, '1', None, '1.0', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
('5ff1e1bb0a720f052300056b', '28400642255', 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ', '10.0', '10.0', '1', '2', '1', '1.0', '28400642255.0', '1', '10.0', '1.0', 'USER_FLAGGED', 'Action not allowed for user and CPG', '5332f5fbe4b03c9a25efd0ba', 'DORITOS SPICY SWEET CHILI SINGLE SERVE', '5332f5fbe4b03c9a25efd0ba', 'DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCED FAT BAG 1 OZ', None, None, 

In [39]:
# create updated receipts tables in the database 
mycursor.execute("USE fetch_dataChallenge")
mycursor.execute("DROP TABLE IF EXISTS brands_updated")
mycursor.execute("CREATE TABLE brands_updated (id VARCHAR(255), barcode VARCHAR(255), category VARCHAR(255), categoryCode VARCHAR(255), cpg_id VARCHAR(255), cpg_ref VARCHAR(255), name VARCHAR(255), topBrand VARCHAR(10))")

# change the data type of the columns to string type
brands['_id'] = brands['_id'].astype(str)  #int(brands['_id'])
brands['barcode'] = brands['barcode'].astype(str) 
brands['topBrand'] = brands['topBrand'].astype(str)

# Insert row by row to Table with its corresponding columns from the receipts_updated dataframe
for i in range(len(brands)):
    sql = "INSERT INTO brands_updated (id, barcode, category, categoryCode, cpg_id, cpg_ref, name, topBrand) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    val = (brands['_id'].iloc[i], brands['barcode'].iloc[i], brands['category'].iloc[i], brands['categoryCode'].iloc[i], brands['cpg_id'].iloc[i], brands['cpg_ref'].iloc[i], brands['name'].iloc[i], brands['topBrand'].iloc[i])
    mycursor.execute(sql, val)
    mydb.commit()


# print the first 10 rows of the brands table
mycursor.execute("SELECT * FROM brands_updated LIMIT 10")
myresult = mycursor.fetchall()

for x in myresult:
  print(x)

('601ac115be37ce2ead437551', '511111019862', 'Baking', 'BAKING', '601ac114be37ce2ead437550', 'Cogs', 'test brand @1612366101024', 'False')
('601c5460be37ce2ead43755f', '511111519928', 'Beverages', 'BEVERAGES', '5332f5fbe4b03c9a25efd0ba', 'Cogs', 'Starbucks', 'False')
('601ac142be37ce2ead43755d', '511111819905', 'Baking', 'BAKING', '601ac142be37ce2ead437559', 'Cogs', 'test brand @1612366146176', 'False')
('601ac142be37ce2ead43755a', '511111519874', 'Baking', 'BAKING', '601ac142be37ce2ead437559', 'Cogs', 'test brand @1612366146051', 'False')
('601ac142be37ce2ead43755e', '511111319917', 'Candy & Sweets', 'CANDY_AND_SWEETS', '5332fa12e4b03c9a25efd1e7', 'Cogs', 'test brand @1612366146827', 'False')
('601ac142be37ce2ead43755b', '511111719885', 'Baking', 'BAKING', '601ac142be37ce2ead437559', 'Cogs', 'test brand @1612366146091', 'False')
('601ac142be37ce2ead43755c', '511111219897', 'Baking', 'BAKING', '601ac142be37ce2ead437559', 'Cogs', 'test brand @1612366146133', 'False')
('5cdad0f5166eb33eb

In [40]:
# select the rows where barcode is not unique
mycursor.execute("SELECT barcode, COUNT(*) FROM brands_updated GROUP BY barcode HAVING COUNT(*) > 1")
myresult = mycursor.fetchall()

for x in myresult:
  print(x)

('511111504788', 2)
('511111305125', 2)
('511111504139', 2)
('511111204923', 2)
('511111605058', 2)
('511111004790', 2)
('511111704140', 2)


### What are the top 5 brands by receipts scanned for most recent month?

In [41]:
# find the most recent top 5 and save the resuls in a temporary table
mycursor.execute("CREATE TEMPORARY TABLE recent_rank5 AS SELECT brands_updated.name, COUNT(*) \
                  FROM receipts_rewardList INNER JOIN brands_updated ON receipts_rewardList.barcode = brands_updated.barcode INNER JOIN receipts_updated ON receipts_updated.id = receipts_rewardList.receipt_id\
                  where SUBSTRING(receipts_updated.date_scanned, 1, 10) >= '2021-01-01' AND SUBSTRING(receipts_updated.date_scanned, 1, 10) <= '2021-02-01'\
                  GROUP BY brands_updated.name ORDER BY COUNT(*) DESC LIMIT 5 ")


mycursor.execute("SELECT * FROM recent_rank5")
myresult = mycursor.fetchall()

for x in myresult:
  print(x)



('Tostitos', 23)
('Swanson', 11)
('Cracker Barrel Cheese', 10)
('Prego', 7)
('Diet Chris Cola', 7)


### How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [42]:
# find out the rank of brand on the last 2 months with a rank coulumn

mycursor.execute(" SELECT brands_updated.name, COUNT(*) \
                    FROM receipts_rewardList INNER JOIN brands_updated ON receipts_rewardList.barcode = brands_updated.barcode INNER JOIN receipts_updated ON receipts_updated.id = receipts_rewardList.receipt_id\
                    where SUBSTRING(receipts_updated.date_scanned, 1, 10) >= '2021-01-01' AND SUBSTRING(receipts_updated.date_scanned, 1, 10) <= '2021-02-01'\
                    GROUP BY brands_updated.name ORDER BY COUNT(*) DESC LIMIT 10 ")



#mycursor.execute("SELECT * FROM recent_rank5")
myresult = mycursor.fetchall()

for x in myresult:
  print(x)


('Tostitos', 23)
('Swanson', 11)
('Cracker Barrel Cheese', 10)
('Diet Chris Cola', 7)
('Prego', 7)
('Pepperidge Farm', 5)
('V8', 4)
('Kettle Brand', 3)
('Quaker', 3)
('Jell-O', 3)


**Those two answers are highly doubtful since the barcode from reciptes cannot match to the barcode from brands. More investigation have to be done**

#### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [43]:
# calculate average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ 
mycursor.execute("SELECT AVG(total_spent) FROM receipts_updated WHERE rewards_receipt_status = 'FINISHED'")
myresult = mycursor.fetchall()


for x in myresult:
  print(x)


(80.85430524938356,)


In [44]:
# calculate average spend from receipts with 'rewardsReceiptStatus’ of ‘Rejected’
mycursor.execute("SELECT AVG(total_spent) FROM receipts_updated WHERE rewards_receipt_status = 'REJECTED'")
myresult = mycursor.fetchall()


for x in myresult:
  print(x)


(23.326055781942017,)


-  It's clear finished recipte has larger average spends(80.8 > 23.3).

#### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [45]:
# calculate total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’
mycursor.execute("SELECT AVG(purchased_Item_Count) FROM receipts_updated WHERE rewards_receipt_status = 'FINISHED'")
myresult = mycursor.fetchall()


for x in myresult:
  print(x)


(15.799227799227799,)


In [46]:
# calculate total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Rejected’
mycursor.execute("SELECT AVG(purchased_Item_Count) FROM receipts_updated WHERE rewards_receipt_status = 'REJECTED'")
myresult = mycursor.fetchall()


for x in myresult:
  print(x)

(2.436619718309859,)


-  It's clear finished recipte has larger average total number of items (15.8 > 2.4).

#### Which brand has the most spend among users who were created within the past 6 months?

- Create a table count number of transactions and totals spends for new users created within the past 6 months.


In [47]:
mycursor.execute("DROP TABLE IF EXISTS topbrand_newUser")
mycursor.execute("CREATE TEMPORARY TABLE topbrand_newUser AS\
                 SELECT brands_updated.name, SUM(receipts_updated.total_spent) as spends, count(*) as transNum\
                  FROM receipts_rewardList INNER JOIN brands_updated ON receipts_rewardList.barcode = brands_updated.barcode INNER JOIN receipts_updated ON receipts_updated.id = receipts_rewardList.receipt_id INNER JOIN  users_updated ON users_updated.id = receipts_updated.user_id\
                  where SUBSTRING(users_updated.created_date, 1, 10) >= '2020-09-01' \
                  GROUP BY brands_updated.name")

mycursor.execute("SELECT * FROM topbrand_newUser")
myresult = mycursor.fetchall()

for x in myresult:
  print(x)

('Swanson', 14907.769775390625, 28)
('V8', 9443.900390625, 2)
('Jell-O', 4754.370193481445, 2)
('Quaker', 32.41999816894531, 1)
('Pepperidge Farm', 14165.8505859375, 3)
('Grey Poupon', 743.7899780273438, 1)
('Diet Chris Cola', 9443.900390625, 2)
('Prego', 9443.900390625, 2)
('Kettle Brand', 2400.9099731445312, 3)
('Cheetos', 4721.9501953125, 1)
('Cracker Barrel Cheese', 5213.770202636719, 4)
('Tostitos', 23812.969787597656, 43)


In [48]:
mycursor.execute("SELECT name, spends FROM topbrand_newUser ORDER BY spends DESC LIMIT 1")


myresult = mycursor.fetchall()

for x in myresult:
  print(x)


('Tostitos', 23812.969787597656)


#### Which brand has the most transactions among users who were created within the past 6 months?

In [49]:
mycursor.execute("SELECT name, transNum FROM topbrand_newUser ORDER BY transNum DESC LIMIT 1")

myresult = mycursor.fetchall()

for x in myresult:
  print(x)


('Tostitos', 43)


### Evaluate Data Quality Issues in the Data Provided

When writing the SQL queries, I find out there are several issues:

-  data issue 1: the same barcode has different names

In [50]:
# find out duplicate rows in the brands_updated.csv
mycursor.execute("SELECT barcode, COUNT(*) FROM brands_updated GROUP BY barcode HAVING COUNT(*) > 1")
myresult = mycursor.fetchall()

for x in myresult:
  print(x)

('511111504788', 2)
('511111305125', 2)
('511111504139', 2)
('511111204923', 2)
('511111605058', 2)
('511111004790', 2)
('511111704140', 2)


In [51]:
# export the required packages as requirement.txt
!pip freeze > requirements.txt
